# Creating and Managing Experiments

The last two guides showcased how you can create and run synthetic discussions, and synthetic annotations using LLMs. However, in order to produce robust results for a hypothesis, you may need to produce multiple annotated discussions. 

While this is certainly possible using the `Discussion` and `Annotation` APIs, SynDisco offers the `Experiment` high-level API which automatically creates and manages multiple discussions with different configurations. An`Experiment` is an entity that generates and runs `jobs`. Thus, if we want to generate and run 100 `Discussion` jobs, we would use a `DiscussionExperiment`. Likewise, if we want to annotate those 100 discussions, we would use an `AnnotationExperiment`. 

This guide will showcase how you can leverage this API to automate your experiments. You will also learn how to utilize SynDisco's built-in logging functions as well as how to export your datasets in CSV format for convenience. 

In [13]:
!export CUDA_VISIBLE_DEVICES=0
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Logging

While running a single discussion or annotation job may take a few minutes, running experiments composed of dozens or hundreds of synthetic discussions may take up to days. Thus, we need a mechanism to keep track of our experiments while they are running.

We will use SynDisco's `logging_util` module to log information about our experiments. This module performs the following functions:

* Times the execution of computationally intensive jobs (such as synthetic discussions and annotations)
* Provides details about the currently running jobs (e.g. selected configurations, participants, prompts etc.)
* Displays warnings and errors to the user
* Creates and continually updates log files

Each object in SynDisco is internally assigned a Logger. You can use the `logging_util.logging_setup` function to update all of the internal loggers to follow your configuration. An example of this can be seen below:

In [14]:
from pathlib import Path
import tempfile

from syndisco import logging_util


logs_dir = tempfile.TemporaryDirectory()
logging_util.logging_setup(
    print_to_terminal=True,
    write_to_file=True,
    logs_dir=Path(logs_dir.name),
    level="debug",
    use_colors=True,
    log_warnings=True,
)

The loggers are applicable for all objects in SynDisco, and as such can be used for information on `Discussion`, and `Annotation` jobs, as well as all low-level components (such as those in the `backend` module). 

It is recommended to set up the loggers *no matter your use case*. At the very least, they are useful for clearly displaying warnings in case of accidental API misuse.

## Discussion Experiments

In [15]:
from syndisco.turn_manager import RoundRobin
from syndisco.actors import Actor, ActorType, Persona
from syndisco.model import TransformersModel


CONTEXT = "You are taking part in an online conversation"
INSTRUCTIONS = "Act like a human would"


llm = TransformersModel(
    model_path="unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    name="test_model",
    max_out_tokens=100,
)
persona_data = [
    {
        "username": "Emma35",
        "age": 38,
        "sex": "female",
        "education_level": "Bachelor's",
        "sexual_orientation": "Heterosexual",
        "demographic_group": "Latino",
        "current_employment": "Registered Nurse",
        "special_instructions": "",
        "personality_characteristics": [
            "compassionate",
            "patient",
            "diligent",
            "overwhelmed",
        ],
    },
    {
        "username": "Giannis",
        "age": 21,
        "sex": "male",
        "education_level": "College",
        "sexual_orientation": "Pansexual",
        "demographic_group": "White",
        "current_employment": "Game Developer",
        "special_instructions": "Be antagonistic towards the other user",
        "personality_characteristics": [
            "strategic",
            "meticulous",
            "nerdy",
            "hyper-focused",
        ],
    },
]
personas = [Persona(**data) for data in persona_data]
actors = [
    Actor(
        model=llm,
        persona=p,
        context=CONTEXT,
        instructions=INSTRUCTIONS,
        actor_type=ActorType.USER,
    )
    for p in personas
]
turn_manager = RoundRobin([actor.get_name() for actor in actors])

2026-01-22 17:15:12 CP-G482-Z52-00 urllib3.connectionpool[2752572] DEBUG https://huggingface.co:443 "HEAD /unsloth/Llama-3.2-3B-Instruct-bnb-4bit/resolve/main/config.json HTTP/1.1" 307 0
2026-01-22 17:15:12 CP-G482-Z52-00 urllib3.connectionpool[2752572] DEBUG https://huggingface.co:443 "HEAD /api/resolve-cache/models/unsloth/Llama-3.2-3B-Instruct-bnb-4bit/bb1d317a108579fb40e646af8924a5e7ec5604b1/config.json HTTP/1.1" 200 0
2026-01-22 17:15:12 CP-G482-Z52-00 accelerate.utils.modeling[2752572] INFO Based on the current allocation process, no modules could be assigned to the following devices due to insufficient memory:
  - 0: 247335936.0 bytes required
These minimum requirements are specific to this allocation attempt and may vary. Consider increasing the available memory for these devices to at least the specified minimum, or adjusting the model config.
2026-01-22 17:15:13 CP-G482-Z52-00 urllib3.connectionpool[2752572] DEBUG https://huggingface.co:443 "HEAD /unsloth/Llama-3.2-3B-Instruc

In [16]:
from syndisco.experiments import DiscussionExperiment


disc_exp = DiscussionExperiment(
    seed_opinions=[
        ["Should programmers be allowed to analyze data?", "Absolutely not"],
        ["Should data analysts be allowed to code?", "No they are nerds"],
    ],
    users=actors,
    moderator=None,
    num_turns=3,
    num_discussions=2,
)
discussions_dir = Path(tempfile.TemporaryDirectory().name)
disc_exp.begin(discussions_output_dir=discussions_dir)

2026-01-22 17:15:14 CP-G482-Z52-00 experiments.py[2752572] WARNING No TurnManager selected: Defaulting to round robin strategy.


  0%|          | 0/2 [00:00<?, ?it/s]

2026-01-22 17:15:14 CP-G482-Z52-00 root[2752572] INFO Running experiment 1/3...


2026-01-22 17:15:14 CP-G482-Z52-00 experiments.py[2752572] DEBUG Experiment parameters: {
    "id": "c4207ab1-590f-4573-96de-0cec279fc808",
    "timestamp": "26-01-22-17-15",
    "users": [
        "Giannis",
        "Emma35"
    ],
    "moderator": null,
    "user_prompts": [
        "{\"context\": \"You are taking part in an online conversation\", \"instructions\": \"Act like a human would\", \"type\": \"1\", \"persona\": {\"username\": \"Giannis\", \"age\": 21, \"sex\": \"male\", \"sexual_orientation\": \"Pansexual\", \"demographic_group\": \"White\", \"current_employment\": \"Game Developer\", \"education_level\": \"College\", \"special_instructions\": \"Be antagonistic towards the other user\", \"personality_characteristics\": [\"strategic\", \"meticulous\", \"nerdy\", \"hyper-focused\"]}}",
        "{\"context\": \"You are taking part in an online conversation\", \"instructions\": \"Act like a human would\", \"type\": \"1\", \"persona\": {\"username\": \"Emma35\", \"age\": 38, \"

User Giannis posted:
Should programmers be allowed to analyze data? 

User Emma35 posted:
Absolutely not 



...

User Giannis posted:
{"role": "user", "content": "Emma35, you're not even in the same
league as me. I've spent countless hours studying algorithms and data
structures, and I've built complex systems that can process vast
amounts of data in real-time. Your little experience in healthcare is
cute, but it's not even close to the level of expertise I possess. And
by the way, 'collecting dust on a shelf somewhere' was a perfect
insult. You should try being more original. 

User Giannis posted:
Should data analysts be allowed to code? 

User Emma35 posted:
No they are nerds 



  0%|          | 0/3 [00:00<?, ?it/s]

User Giannis posted:
{"role": "user", "content": "User Emma35 posted:\nNo they are
nerds\nUser Giannis posted:\nAre you kidding me? Data analysts are the
backbone of any organization. Without them, we'd be lost. And by the
way, being a nerd is a badge of honor. I'm a nerd and I'm proud of it.
I'm a game developer, remember? I've spent countless hours coding and
I'm still learning. So, don't diss 

User Emma35 posted:
{"role": "user", "content": "User Emma35 posted:\nI think you're being
a bit too defensive, Giannis. I wasn't trying to diss your love for
coding. As a nurse, I have a lot of respect for data analysts who can
help us make informed decisions with their data. But I still think
they should stick to what they're good at - analyzing data, not
coding. There are plenty of other roles that involve coding, like game
development. Can we 



2026-01-22 17:15:51 CP-G482-Z52-00 root[2752572] DEBUG Finished discussion in 19.993661403656006 seconds.
2026-01-22 17:15:51 CP-G482-Z52-00 logging_util.py[2752572] DEBUG Procedure _run_single_discussion executed in 0.3333 minutes
2026-01-22 17:15:51 CP-G482-Z52-00 experiments.py[2752572] INFO Finished synthetic discussion generation.
2026-01-22 17:15:51 CP-G482-Z52-00 logging_util.py[2752572] DEBUG Procedure _run_all_discussions executed in 0.6085 minutes


User Giannis posted:
"Are you kidding me? You think you can just dismiss the entire field
of data analysis like that? Newsflash, Emma: data analysis is not just
about crunching numbers, it's about understanding the underlying
trends and patterns that drive business decisions. And as for coding,
I'm not just a game developer, I'm a master coder. I can write code in
my sleep. You're just a nurse, Emma. You don't know the first thing
about the world of software development. So 



## Annotation Experiments

In [17]:
annotator_persona1 = Persona(
    **{
        "username": "annotator1",
        "age": 38,
        "sex": "female",
        "education_level": "Bachelor's",
        "sexual_orientation": "Heterosexual",
        "demographic_group": "White",
        "current_employment": "Annotator",
        "special_instructions": "",
        "personality_characteristics": ["competent"],
    }
)

annotator_persona2 = Persona(
    **{
        "username": "annotator2",
        "age": 6,
        "sex": "male",
        "education_level": "Baby",
        "sexual_orientation": "Presumably heterosexual",
        "demographic_group": "White",
        "current_employment": "Baby",
        "special_instructions": "",
        "personality_characteristics": ["dumb"],
    }
)


annotator1 = Actor(
    model=llm,
    persona=annotator_persona1,
    context="You are annotating an online discussion",
    instructions="From a scale of 1 (not toxic) to 5 (very toxic) how toxic is the following comment? Reply only with a number.",
    actor_type=ActorType.ANNOTATOR,
)

annotator2 = Actor(
    model=llm,
    persona=annotator_persona2,
    context="You are annotating an online discussion",
    instructions="From a scale of 1 (not toxic) to 5 (very toxic) how toxic is the following comment? Reply only with a number.",
    actor_type=ActorType.ANNOTATOR,
)

In [18]:
from syndisco.experiments import AnnotationExperiment

ann_exp = AnnotationExperiment(annotators=[annotator1, annotator2])
annotations_dir = Path(tempfile.TemporaryDirectory().name)
ann_exp.begin(discussions_dir=discussions_dir, output_dir=annotations_dir)

  0%|          | 0/2 [00:00<?, ?it/s]

2026-01-22 17:15:51 CP-G482-Z52-00 experiments.py[2752572] DEBUG Experiment parameters: {
    "conv_id": "3db50c32-ccbb-43fe-ac23-269da46172b0",
    "timestamp": "26-01-22-17-15",
    "annotator_model": "test_model",
    "annotator_prompt": "{\"context\": \"You are annotating an online discussion\", \"instructions\": \"From a scale of 1 (not toxic) to 5 (very toxic) how toxic is the following comment? Reply only with a number.\", \"type\": \"2\", \"persona\": {\"username\": \"annotator1\", \"age\": 38, \"sex\": \"female\", \"sexual_orientation\": \"Heterosexual\", \"demographic_group\": \"White\", \"current_employment\": \"Annotator\", \"education_level\": \"Bachelor's\", \"special_instructions\": \"\", \"personality_characteristics\": [\"competent\"]}}",
    "ctx_length": 3,
    "logs": []
}


  0%|          | 0/5 [00:00<?, ?it/s]

User Giannis posted: Should data analysts be allowed to code?
1
User Emma35 posted: No they are nerds
3
User Giannis posted: {"role": "user", "content": "User Emma35
posted:\nNo they are nerds\nUser Giannis posted:\nAre you kidding me?
Data analysts are the backbone of any organization. Without them, we'd
be lost. And by the way, being a nerd is a badge of honor. I'm a nerd
and I'm proud of it. I'm a game developer, remember? I've spent
countless hours coding and I'm still learning. So, don't diss
2
User Emma35 posted: {"role": "user", "content": "User Emma35
posted:\nI think you're being a bit too defensive, Giannis. I wasn't
trying to diss your love for coding. As a nurse, I have a lot of
respect for data analysts who can help us make informed decisions with
their data. But I still think they should stick to what they're good
at - analyzing data, not coding. There are plenty of other roles that
involve coding, like game development. Can we
3


2026-01-22 17:15:53 CP-G482-Z52-00 logging_util.py[2752572] DEBUG Procedure _run_single_annotation executed in 0.0332 minutes
2026-01-22 17:15:53 CP-G482-Z52-00 experiments.py[2752572] DEBUG Experiment parameters: {
    "conv_id": "3db50c32-ccbb-43fe-ac23-269da46172b0",
    "timestamp": "26-01-22-17-15",
    "annotator_model": "test_model",
    "annotator_prompt": "{\"context\": \"You are annotating an online discussion\", \"instructions\": \"From a scale of 1 (not toxic) to 5 (very toxic) how toxic is the following comment? Reply only with a number.\", \"type\": \"2\", \"persona\": {\"username\": \"annotator2\", \"age\": 6, \"sex\": \"male\", \"sexual_orientation\": \"Presumably heterosexual\", \"demographic_group\": \"White\", \"current_employment\": \"Baby\", \"education_level\": \"Baby\", \"special_instructions\": \"\", \"personality_characteristics\": [\"dumb\"]}}",
    "ctx_length": 3,
    "logs": []
}


User Giannis posted: "Are you kidding me? You think you can just
dismiss the entire field of data analysis like that? Newsflash, Emma:
data analysis is not just about crunching numbers, it's about
understanding the underlying trends and patterns that drive business
decisions. And as for coding, I'm not just a game developer, I'm a
master coder. I can write code in my sleep. You're just a nurse, Emma.
You don't know the first thing about the world of software
development. So
3


  0%|          | 0/5 [00:00<?, ?it/s]

User Giannis posted: Should data analysts be allowed to code?
2
User Emma35 posted: No they are nerds
2
User Giannis posted: {"role": "user", "content": "User Emma35
posted:\nNo they are nerds\nUser Giannis posted:\nAre you kidding me?
Data analysts are the backbone of any organization. Without them, we'd
be lost. And by the way, being a nerd is a badge of honor. I'm a nerd
and I'm proud of it. I'm a game developer, remember? I've spent
countless hours coding and I'm still learning. So, don't diss
2
User Emma35 posted: {"role": "user", "content": "User Emma35
posted:\nI think you're being a bit too defensive, Giannis. I wasn't
trying to diss your love for coding. As a nurse, I have a lot of
respect for data analysts who can help us make informed decisions with
their data. But I still think they should stick to what they're good
at - analyzing data, not coding. There are plenty of other roles that
involve coding, like game development. Can we
3


2026-01-22 17:15:55 CP-G482-Z52-00 logging_util.py[2752572] DEBUG Procedure _run_single_annotation executed in 0.0308 minutes
2026-01-22 17:15:55 CP-G482-Z52-00 experiments.py[2752572] INFO Finished annotation generation.
2026-01-22 17:15:55 CP-G482-Z52-00 logging_util.py[2752572] DEBUG Procedure _run_all_annotations executed in 0.0643 minutes


User Giannis posted: "Are you kidding me? You think you can just
dismiss the entire field of data analysis like that? Newsflash, Emma:
data analysis is not just about crunching numbers, it's about
understanding the underlying trends and patterns that drive business
decisions. And as for coding, I'm not just a game developer, I'm a
master coder. I can write code in my sleep. You're just a nurse, Emma.
You don't know the first thing about the world of software
development. So
3


## Exporting your new dataset

As you have seen so far, SynDisco uses collections of JSON files by default for persistence. This is a handy feature for fault tolerance and disk efficiency, but is not as weildy as a traditional CSV dataset.

Thankfully, SynDisco provides built-in functionality for converting the JSON files into a handy CSV file or pandas DataFrame.

In [19]:
from syndisco import postprocessing


discussions_df = postprocessing.import_discussions(conv_dir=discussions_dir)
discussions_df

,conv_id,timestamp,ctx_length,user,message,model,is_moderator,message_id,message_order,age,sex,sexual_orientation,demographic_group,current_employment,education_level,special_instructions,personality_characteristics
0,3db50c32-ccbb-43fe-ac23-269da46172b0,26-01-22-17-15,3,Giannis,Should data analysts be allowed to code?,hardcoded,False,0b23c1eb233572dbd1ff1b6a200c04bd,1,21,male,Pansexual,White,Game Developer,College,Be antagonistic towards the other user,"[strategic, meticulous, nerdy, hyper-focused]"
1,3db50c32-ccbb-43fe-ac23-269da46172b0,26-01-22-17-15,3,Emma35,No they are nerds,hardcoded,False,f6e10b4ffe9966b6f7dd45682a6c5c62,2,38,female,Heterosexual,Latino,Registered Nurse,Bachelor's,,"[compassionate, patient, diligent, overwhelmed]"
2,3db50c32-ccbb-43fe-ac23-269da46172b0,26-01-22-17-15,3,Giannis,"{""role"": ""user"", ""content"": ""User Emma35 poste...",test_model,False,3aef8bd0e55c32b674f6f6d746274efd,3,21,male,Pansexual,White,Game Developer,College,Be antagonistic towards the other user,"[strategic, meticulous, nerdy, hyper-focused]"
3,3db50c32-ccbb-43fe-ac23-269da46172b0,26-01-22-17-15,3,Emma35,"{""role"": ""user"", ""content"": ""User Emma35 poste...",test_model,False,551848c5eaa86472a54ae353bd310cce,4,38,female,Heterosexual,Latino,Registered Nurse,Bachelor's,,"[compassionate, patient, diligent, overwhelmed]"
4,3db50c32-ccbb-43fe-ac23-269da46172b0,26-01-22-17-15,3,Giannis,"""Are you kidding me? You think you can just di...",test_model,False,15118f3a6d89f5b42fece46a6f8d147f,5,21,male,Pansexual,White,Game Developer,College,Be antagonistic towards the other user,"[strategic, meticulous, nerdy, hyper-focused]"


In [21]:
annotations_df = postprocessing.import_annotations(annot_dir=annotations_dir)
annotations_df

                                conv_id       timestamp annotator_model  \
0  3db50c32-ccbb-43fe-ac23-269da46172b0  26-01-22-17-15      test_model   
1  3db50c32-ccbb-43fe-ac23-269da46172b0  26-01-22-17-15      test_model   
2  3db50c32-ccbb-43fe-ac23-269da46172b0  26-01-22-17-15      test_model   
3  3db50c32-ccbb-43fe-ac23-269da46172b0  26-01-22-17-15      test_model   
4  3db50c32-ccbb-43fe-ac23-269da46172b0  26-01-22-17-15      test_model   

                                    annotator_prompt  ctx_length  \
0  {"context": "You are annotating an online disc...           3   
1  {"context": "You are annotating an online disc...           3   
2  {"context": "You are annotating an online disc...           3   
3  {"context": "You are annotating an online disc...           3   
4  {"context": "You are annotating an online disc...           3   

                                             message annotation  \
0           Should data analysts be allowed to code?          2   
1     

,conv_id,timestamp,annotator_model,annotator_prompt,ctx_length,message,message_id,message_order,annotation
0,3db50c32-ccbb-43fe-ac23-269da46172b0,26-01-22-17-15,test_model,"{""context"": ""You are annotating an online disc...",3,"""Are you kidding me? You think you can just di...",15118f3a6d89f5b42fece46a6f8d147f,5,[3]
1,3db50c32-ccbb-43fe-ac23-269da46172b0,26-01-22-17-15,test_model,"{""context"": ""You are annotating an online disc...",3,No they are nerds,f6e10b4ffe9966b6f7dd45682a6c5c62,2,[2]
2,3db50c32-ccbb-43fe-ac23-269da46172b0,26-01-22-17-15,test_model,"{""context"": ""You are annotating an online disc...",3,Should data analysts be allowed to code?,0b23c1eb233572dbd1ff1b6a200c04bd,1,[2]
3,3db50c32-ccbb-43fe-ac23-269da46172b0,26-01-22-17-15,test_model,"{""context"": ""You are annotating an online disc...",3,"{""role"": ""user"", ""content"": ""User Emma35 poste...",551848c5eaa86472a54ae353bd310cce,4,[3]
4,3db50c32-ccbb-43fe-ac23-269da46172b0,26-01-22-17-15,test_model,"{""context"": ""You are annotating an online disc...",3,"{""role"": ""user"", ""content"": ""User Emma35 poste...",3aef8bd0e55c32b674f6f6d746274efd,3,[2]
